In [22]:
import pandas as pd

In [23]:
import mysql.connector as mysql

In [24]:
def connect():
    conn=mysql.connect(
        host='127.0.0.1',
        user='',
        password='',
        database=''
    )
    cursor=conn.cursor()
    return conn,cursor

In [25]:
conn,cursor=connect()

In [26]:
query='''
select basket_string from shop_marketbasket;
'''

In [27]:
cursor.execute(query)

In [28]:
data=cursor.fetchall()

In [29]:
cursor.close()
conn.close()

In [30]:
data

[('Kaapicino,Combo-south Indian Filter Kaapi 150 ML + Papparoti,Espresso',),
 ('Coconut Natkhatai (With Egg),Hot Chocolate,Baked Vada Pav',),
 ('Calzones Veg,Espresso,Combo-south Indian Filter Kaapi 150 ML + Papparoti',),
 ('Madagascar Hot Chocolate,Plain Nankhatai Veg,Baked Vada Pav',),
 ('Madagascar Hot Chocolate,Espresso,Baked Vada Pav',),
 ('Combo-south Indian Filter Kaapi 150 ML + Papparoti,Espresso,Baked Vada Pav,Espresso',)]

In [31]:
df=pd.DataFrame(data,columns=['text'])

In [32]:
df

,text
0,"Kaapicino,Combo-south Indian Filter Kaapi 150 ..."
1,"Coconut Natkhatai (With Egg),Hot Chocolate,Bak..."
2,"Calzones Veg,Espresso,Combo-south Indian Filte..."
3,"Madagascar Hot Chocolate,Plain Nankhatai Veg,B..."
4,"Madagascar Hot Chocolate,Espresso,Baked Vada Pav"
5,Combo-south Indian Filter Kaapi 150 ML + Pappa...


In [33]:
df=df.text.str.split(',',expand=True)

In [41]:
type(df.values[0][0])

str

In [37]:
from apyori import apriori

In [46]:
results = list(apriori(df.values))

In [44]:
df.replace({None:""},inplace=True)

In [45]:
df

,0,1,2,3
0,Kaapicino,Combo-south Indian Filter Kaapi 150 ML + Pappa...,Espresso,
1,Coconut Natkhatai (With Egg),Hot Chocolate,Baked Vada Pav,
2,Calzones Veg,Espresso,Combo-south Indian Filter Kaapi 150 ML + Pappa...,
3,Madagascar Hot Chocolate,Plain Nankhatai Veg,Baked Vada Pav,
4,Madagascar Hot Chocolate,Espresso,Baked Vada Pav,
5,Combo-south Indian Filter Kaapi 150 ML + Pappa...,Espresso,Baked Vada Pav,Espresso


In [49]:
from pprint import pprint

In [55]:
results=list(results)

In [66]:
results[0[2][0][2]

0.8333333333333334

In [68]:
sorted_data=sorted(results,key=lambda x:x[2][0][2],reverse=True)

In [69]:
sorted_data

[RelationRecord(items=frozenset({''}), support=0.8333333333333334, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({''}), confidence=0.8333333333333334, lift=1.0)]),
 RelationRecord(items=frozenset({'Baked Vada Pav'}), support=0.6666666666666666, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Baked Vada Pav'}), confidence=0.6666666666666666, lift=1.0)]),
 RelationRecord(items=frozenset({'Espresso'}), support=0.6666666666666666, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Espresso'}), confidence=0.6666666666666666, lift=1.0)]),
 RelationRecord(items=frozenset({'Combo-south Indian Filter Kaapi 150 ML + Papparoti'}), support=0.5, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Combo-south Indian Filter Kaapi 150 ML + Papparoti'}), confidence=0.5, lift=1.0)]),
 RelationRecord(items=frozenset({'', 'Baked Vada Pav'}), support=0.5, ordered_statistics=[

In [71]:
def inspect(results):
    rules = []
    for result in results:
        for ordered_stat in result.ordered_statistics:
            rules.append({
                'items_base': tuple(ordered_stat.items_base),
                'items_add': tuple(ordered_stat.items_add),
                'support': result.support,
                'confidence': ordered_stat.confidence,
                'lift': ordered_stat.lift
            })
    return pd.DataFrame(rules)

# Extract the rules into a DataFrame
rules_df = inspect(results)

# Sort the rules by confidence in descending order and select the top 2
top_rules = rules_df.sort_values(by='confidence', ascending=False).head()

# Display the top 2 rules in the format item1 -> item2, item3
for index, rule in top_rules.iterrows():
    items_base = ', '.join(rule['items_base'])
    items_add = ', '.join(rule['items_add'])
    print(f"{items_base} -> {items_add}")

Espresso, Combo-south Indian Filter Kaapi 150 ML + Papparoti, Kaapicino -> 
, Madagascar Hot Chocolate -> Baked Vada Pav
Kaapicino -> , Espresso
, Kaapicino -> Espresso


In [72]:
rules_df

,items_base,items_add,support,confidence,lift
0,(),"(,)",0.833333,0.833333,1.0
1,(),"(Baked Vada Pav,)",0.666667,0.666667,1.0
2,(),"(Calzones Veg,)",0.166667,0.166667,1.0
3,(),"(Coconut Natkhatai (With Egg),)",0.166667,0.166667,1.0
4,(),(Combo-south Indian Filter Kaapi 150 ML + Papp...,0.500000,0.500000,1.0
...,...,...,...,...,...
282,"(Espresso, Kaapicino)","(, Combo-south Indian Filter Kaapi 150 ML + Pa...",0.166667,1.000000,3.0
283,"(, Espresso, Combo-south Indian Filter Kaapi 1...","(Kaapicino,)",0.166667,0.500000,3.0
284,"(, Combo-south Indian Filter Kaapi 150 ML + Pa...","(Espresso,)",0.166667,1.000000,1.5
285,"(, Espresso, Kaapicino)",(Combo-south Indian Filter Kaapi 150 ML + Papp...,0.166667,1.000000,2.0


In [77]:
rules_df.sort_values(by='confidence', ascending=False).head(5)

,items_base,items_add,support,confidence,lift
286,"(Espresso, Combo-south Indian Filter Kaapi 150...","(,)",0.166667,1.0,1.2
105,"(, Madagascar Hot Chocolate)","(Baked Vada Pav,)",0.333333,1.0,1.5
152,"(Kaapicino,)","(, Espresso)",0.166667,1.0,2.0
154,"(, Kaapicino)","(Espresso,)",0.166667,1.0,1.5
155,"(Espresso, Kaapicino)","(,)",0.166667,1.0,1.2


In [79]:
# Display the top 2 rules in the format item1 -> item2, item3
for index, rule in top_rules.iterrows():
    items_base = ', '.join(rule['items_base'])
    items_add = ', '.join(rule['items_add'])
    print(f"{items_base} -> {items_add} confidence={rule['confidence']}\n")

Espresso, Combo-south Indian Filter Kaapi 150 ML + Papparoti, Kaapicino ->  confidence=1.0

, Madagascar Hot Chocolate -> Baked Vada Pav confidence=1.0

Kaapicino -> , Espresso confidence=1.0

, Kaapicino -> Espresso confidence=1.0



In [103]:
data=[]

In [104]:
for index, rule in top_rules.iterrows():
    if '' in rule['items_add'] and len(rule['items_add'])==1:
        continue
    data.append({
        'if_items':base,
        'then_items':rule['items_add'],
        'confidence':rule['confidence']
    })

In [105]:
data

[{'if_items': None, 'then_items': ('Baked Vada Pav',), 'confidence': 1.0},
 {'if_items': None, 'then_items': ('', 'Espresso'), 'confidence': 1.0},
 {'if_items': None, 'then_items': ('Espresso',), 'confidence': 1.0}]

In [ ]:
# #######################

def connect():
    conn=mysql.connect(
        host='127.0.0.1',
        user='',
        password='',
        database=''
    )
    cursor=conn.cursor()
    return conn,cursor

conn,cursor=connect()
query='''
select basket_string from shop_marketbasket;
'''
cursor.execute(query)
data=cursor.fetchall()
cursor.close()
conn.close()
df=pd.DataFrame(data,columns=['text'])
df=df.text.str.split(',',expand=True)
results = list(apriori(df.values))
df.replace({None:""},inplace=True)

def inspect(results):
    rules = []
    for result in results:
        for ordered_stat in result.ordered_statistics:
            rules.append({
                'items_base': tuple(ordered_stat.items_base),
                'items_add': tuple(ordered_stat.items_add),
                'support': result.support,
                'confidence': ordered_stat.confidence,
                'lift': ordered_stat.lift
            })
    return pd.DataFrame(rules)

# Extract the rules into a DataFrame
rules_df = inspect(results)

# Sort the rules by confidence in descending order and select the top 2
top_rules = rules_df.sort_values(by='confidence', ascending=False).head()
data=[]
for index, rule in top_rules.iterrows():
    if '' in rule['items_add'] and len(rule['items_add'])==1:
        continue
    data.append({
        'if_items':base,
        'then_items':rule['items_add'],
        'confidence':rule['confidence']
    })